# Read Data from CSV files

In [3]:
import numpy as np
import pandas as pd
import math

boston = pd.read_csv("Short_Term_Rentals_boston.csv")
print(boston.shape)
print(boston .columns.values)

cambridge = pd.read_csv("Short_Term_Rentals_cambridge.csv")
print(cambridge.shape)
print(cambridge.columns.values)

boston_addr = boston['sam_address'].values


cambridge_points = cambridge[['Latitude','Longitude','Total Bedrooms','Rented Bedrooms']]


(397948, 17)
['sam_address_id' 'issued_registration' 'sam_address'
 'home-share eligible' 'limited-share eligible' 'owner-adjacent eligible'
 'owners_current_license_types' 'income restricted' 'problem property'
 'problem property owner' 'open violation count'
 'violations in the last 6 months' 'legally restricted'
 'unit owner-occupied' 'building owner-occupied' 'units in building'
 'building single owner']
(281, 25)
['ID' 'Issue Date' 'Status' 'Location' 'Latitude' 'Longitude'
 'Short Term Rental Type' 'Property Type'
 'Property Type Additional Description' 'Condo Association'
 'Total Bedrooms' 'Rented Bedrooms' 'Maximum Renter Capacity' 'Kitchen'
 'Bathrooms' 'Airbnb' 'HomeAway' 'FlipKey' 'VRBO' 'Craigslist'
 'Couch Surfing' 'Boston Rentals' 'Other' 'Other Platform Descriptions'
 'All Rental Services']


# Transfer address in boston dataset into latitude and longitude

### I just use the first 500 data entries in the dataset, There are too much data entries. Because of the time limitation, I did not transfer all of them

In [4]:

import geocoder

size = 500
 
def getGeoForAddress(address):
    g = geocoder.arcgis(address+",Boston,MA")
    cor = g.latlng
    #print(cor)
    return cor

boston_points=[]

for i in boston_addr[:size]:
    boston_points.append(getGeoForAddress(i))

print(len(boston_points))
print(boston_points[:5])
boston_df = pd.DataFrame(boston_points, columns = ["Latitude", "Longitude"])      

500
[[42.342186102948965, -71.05503830125812], [42.296690567574636, -71.08580699835521], [42.314748171793006, -71.0931526604247], [42.314377944226344, -71.09188852247681], [42.31451688536563, -71.09247567499565]]


# Draw an example graph of Boston

In [5]:
import folium

latitude = 42.3601
longitude = -71.0589

# Create map and display it
bos_map = folium.Map(location=[latitude, longitude], zoom_start=12)

# Display the map of boston
bos_map

# Label all the houses on the map

In [11]:

# Instantiate a feature group for the incidents in the dataframe
incidents = folium.map.FeatureGroup()


# Loop through the 200 crimes and add each to the incidents feature group

for lat, lng, in zip(cambridge_points['Latitude'], cambridge_points['Longitude']):
    incidents.add_child(
        folium.CircleMarker(
            [lat, lng],
            radius=5, # define how big you want the circle markers to be
            color='yellow',
            fill=True,
            fill_color='red',
            fill_opacity=0.4
        )
    )


for lat, lng, in zip(boston_df['Latitude'], boston_df['Longitude']):
    incidents.add_child(
        folium.CircleMarker(
            [lat, lng],
            radius=5, # define how big you want the circle markers to be
            color='yellow',
            fill=True,
            fill_color='red',
            fill_opacity=0.4
        )
    )

bos_map = folium.Map(location=[latitude, longitude], zoom_start=12)
bos_map.add_child(incidents)


# Use MarkerCluster to automatically show numbers of short-term rent in an area

### Note*: If you Zoom in/Zoom out the map, the clusters will automatically change

In [16]:
# create a mark cluster object
from folium.plugins import MarkerCluster
bos_map = folium.Map(location=[latitude, longitude], zoom_start=12)
marker_cluster = MarkerCluster().add_to(bos_map)

# add data point to the mark cluster
for lat, lng in zip(boston_df['Latitude'], boston_df['Longitude']):
    folium.Marker(
        location=[lat, lng],
        icon=None,
        #popup=label,
    ).add_to(marker_cluster)
    
for lat, lng in zip(cambridge_points['Latitude'], cambridge_points['Longitude']):
    folium.Marker(
        location=[lat, lng],
        icon=None,
        #popup=label,
    ).add_to(marker_cluster)

# add marker_cluster to map
bos_map.add_child(marker_cluster)